In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
st = time.time()

# data is assumed to be [input_params, output_param]

# change the file name if using a different system
data = np.loadtxt('/Users/Sandesh/quantum-ml/ndot/ml/data/two_dot.txt')

# data for first dot
data = data[:,:-1]

# data randomly permuted to improve learning rate
data = np.random.permutation(data)
n_tot = data.shape[0]
n_train = int(0.8*n_tot)
n_test = n_tot - n_train

# count the number of states, +1 because 0 is also a state
charge_max = int(np.max(data[:,-1]))
n_out = charge_max + 1
# there is only one output for a single dot, the charge state
# represnted as prob vector

# input parameters
n_inp = data.shape[1] - 1

print("Number of inputs:", n_inp)
print("Number of charge states:", n_out)

x_train_data = data[:n_train,:-1].reshape((n_train,n_inp))
#generate the prob vector
y_train_data = np.zeros((len(x_train_data),n_out))
for i in range(len(x_train_data)):
    state = int(data[i,n_inp])
    y_train_data[i,state] = 1.0

x_test_data = data[n_train:,:-1].reshape((n_test,n_inp))
y_test_data = np.zeros((len(x_test_data),n_out))
for i in range(len(x_test_data)):
    state = int(data[n_train+i,n_inp])
    y_test_data[i,state] = 1.0

print("Total, Training, Test")
print(n_tot, n_train, n_test)

x = tf.placeholder(tf.float32,[None,n_inp])

W = tf.Variable(tf.zeros([n_inp,n_out]))
b = tf.Variable(tf.zeros([n_out]))

y = tf.nn.softmax(tf.matmul(x, W) + b)

y_ = tf.placeholder(tf.float32,[None,n_out])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(10).minimize(cross_entropy)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

N_train = 50000
for _ in range(N_train):
    if (_ % (N_train/10) == 0):
        print("Shh. The Machine is Learning! : ",int(_ * 100/ N_train),"%")
    (batch_xs, batch_ys) = x_train_data,y_train_data
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

print("\nCompleted in",time.time()-st,"seconds.")

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("\nAccuracy")
print(sess.run(accuracy, feed_dict={x: x_test_data, y_: y_test_data}))

Number of inputs: 3
Number of charge states: 6
Total, Training, Test
2500 2000 500
Shh. The Machine is Learning! :  0 %
Shh. The Machine is Learning! :  10 %
Shh. The Machine is Learning! :  20 %
Shh. The Machine is Learning! :  30 %
Shh. The Machine is Learning! :  40 %
Shh. The Machine is Learning! :  50 %
Shh. The Machine is Learning! :  60 %
Shh. The Machine is Learning! :  70 %
Shh. The Machine is Learning! :  80 %
Shh. The Machine is Learning! :  90 %

Completed in 123.83007097244263 seconds.

Accuracy
0.792


In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib

z = tf.argmax(y,1)
z_corr = tf.argmax(y_,1)
output_model = sess.run([z,z_corr],{x:x_test_data,y_: y_test_data})

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_test_data[:,0],x_test_data[:,1],output_model[0],marker='^',color='b')
ax.scatter(x_test_data[:,0],x_test_data[:,1],output_model[1],marker='+',color='r')


Using matplotlib backend: MacOSX


In [ ]:
%matplotlib auto